In [ ]:
!pip install --upgrade google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.27.0
    Uninstalling google-auth-2.27.0:
      Successfully uninstalled google-auth-2.27.0
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.155.0
    Uninstalling google-api-python-client-2.155.0:
      Successfully uninstalled google-api-python-client-2.155.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.37.0 which is incompatible.


In [1]:
from google.colab import auth
import re

from googleapiclient.discovery import build
from google.auth import default

In [3]:
try:
    auth.authenticate_user()  # Prompts for user login
    creds, _ = default()
    docs_service = build('docs', 'v1', credentials=creds)
    print("Successfully authenticated with Google Docs API.")
except Exception as e:
    print("Error during authentication:", str(e))

Successfully authenticated with Google Docs API.


In [4]:
markdown_text = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

---
Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""


In [6]:
def parse_markdown(md_text):
    """
    Very basic line-by-line parser to identify:
    - Headings (#, ##, ###)
    - Bullet points (*, -)
    - Checkboxes (- [ ])
    - Mentions (@name)
    - Footer
    Returns a list of parsed elements with type and text.
    """
    # Split into lines
    lines = md_text.split("\n")

    # Remove empty lines at start/end
    # (Optional, depends on your data)
    while lines and not lines[0].strip():
        lines.pop(0)
    while lines and not lines[-1].strip():
        lines.pop()

    parsed_elements = []
    in_footer = False

    for line in lines:
        line_stripped = line.strip()

        # Detect the footer (starting with "---")
        # We apply a special style at the end
        if line_stripped.startswith("---"):
            in_footer = True
            # Skip the line with '---'
            continue

        if in_footer:
            # Treat everything after '---' as footer lines
            parsed_elements.append({"type": "footer", "text": line_stripped})
            continue

        # Heading 1 (# )
        if line_stripped.startswith("# "):
            parsed_elements.append({"type": "heading1", "text": line_stripped[2:].strip()})
            continue

        # Heading 2 (## )
        if line_stripped.startswith("## "):
            parsed_elements.append({"type": "heading2", "text": line_stripped[3:].strip()})
            continue

        # Heading 3 (### )
        if line_stripped.startswith("### "):
            parsed_elements.append({"type": "heading3", "text": line_stripped[4:].strip()})
            continue

        # Bullet or checkbox
        if line_stripped.startswith("- "):
            # Check for checkbox pattern: "- [ ]"
            if re.match(r"- \[ \]", line_stripped):
                # e.g. "- [ ] @sarah: do something"
                text = line_stripped[5:].strip()
                parsed_elements.append({"type": "checkbox", "text": text})
            else:
                # Normal bullet
                text = line_stripped[2:].strip()
                parsed_elements.append({"type": "bullet", "text": text})
            continue

        # Bullet with "* "
        if line_stripped.startswith("* "):
            text = line_stripped[2:].strip()
            parsed_elements.append({"type": "bullet", "text": text})
            continue

        # Nested bullet (indentation)
        # (We will keep track of indentation level for nesting)
        match_star = re.match(r"(\s+)\*", line)
        match_dash = re.match(r"(\s+)-", line)
        if match_star or match_dash:
            # Indentation is number of spaces
            indentation = len(match_star.group(1)) if match_star else len(match_dash.group(1))
            text = line.strip(" *-")
            parsed_elements.append({
                "type": "bullet",
                "text": text,
                "indent": indentation
            })
            continue

        # If not matched anything else, treat as normal text
        if line_stripped:
            parsed_elements.append({"type": "text", "text": line_stripped})

    return parsed_elements

def find_mentions(text):
    """
    Find occurrences of '@something' in the text.
    Return a list of (start_index, end_index).
    """
    mentions = []
    for match in re.finditer(r"(@[A-Za-z0-9_]+)", text):
        mentions.append((match.start(), match.end()))
    return mentions

In [7]:
def build_requests(parsed_elements):
    """
    Convert the parsed markdown elements into an array of requests
    for the Docs API. We insert text with the correct style,
    then handle bullets, checkboxes, and mention highlighting.
    """

    requests = []
    # Keep track of current insertion index
    # We'll append text sequentially and track positions as we go.
    current_index = 1  # Document starts at index 1 for content

    # Helper function to insert text with optional style
    def insert_text_request(text, style_type=None):
        return {
            "insertText": {
                "location": {"index": current_index},
                "text": text
            }
        }

    # Helper function to update paragraph style (for headings, etc.)
    def update_paragraph_style_request(start_idx, end_idx, style_type):
        return {
            "updateParagraphStyle": {
                "range": {
                    "startIndex": start_idx,
                    "endIndex": end_idx
                },
                "paragraphStyle": {
                    "namedStyleType": style_type
                },
                "fields": "namedStyleType"
            }
        }

    # Helper function to update text style for mention
    def update_text_style_request(start_idx, end_idx, bold=False, color=None):
        text_style = {}
        if bold:
            text_style["bold"] = True
        if color:
            text_style["foregroundColor"] = {
                "color": {
                    "rgbColor": color
                }
            }
        return {
            "updateTextStyle": {
                "range": {
                    "startIndex": start_idx,
                    "endIndex": end_idx
                },
                "textStyle": text_style,
                "fields": ",".join(text_style.keys())
            }
        }

    # We'll track ranges for bullet paragraphs and checkbox paragraphs
    paragraph_ranges = []
    footer_started_index = None

    for element in parsed_elements:
        text = element.get("text", "")
        el_type = element.get("type", "text")
        indent = element.get("indent", 0)

        if not text:
            continue

        # Always add a newline after each element
        # so each item is its own paragraph for easy styling.
        text_to_insert = text + "\n"

        # Insert the text
        requests.append(insert_text_request(text_to_insert))

        start_idx = current_index
        end_idx = current_index + len(text_to_insert)
        current_index = end_idx  # Move the index forward

        # If it's a heading, apply paragraph style
        if el_type == "heading1":
            requests.append(update_paragraph_style_request(start_idx, end_idx, "HEADING_1"))
        elif el_type == "heading2":
            requests.append(update_paragraph_style_request(start_idx, end_idx, "HEADING_2"))
        elif el_type == "heading3":
            requests.append(update_paragraph_style_request(start_idx, end_idx, "HEADING_3"))

        # If it's a normal bullet or checkbox, we record the paragraph range
        if el_type in ["bullet", "checkbox"]:
            paragraph_ranges.append((start_idx, end_idx, el_type, indent))

        # If it's footer, we will style it differently at the end
        if el_type == "footer":
            if footer_started_index is None:
                footer_started_index = start_idx

        # Find mentions (@someone) and make them bold and colored
        mentions = find_mentions(text)
        for m_start, m_end in mentions:
            # Adjust for insertion offset
            actual_start = start_idx + m_start
            actual_end = start_idx + m_end
            requests.append(update_text_style_request(actual_start, actual_end, bold=True, color={"red": 0.0, "green": 0.2, "blue": 0.8}))

    # After we've inserted all text, we can create bullet or checkbox for each bullet paragraph
    for (p_start, p_end, bullet_type, indent) in paragraph_ranges:
        if bullet_type == "checkbox":
            requests.append({
                "createParagraphBullets": {
                    "range": {
                        "startIndex": p_start,
                        "endIndex": p_end
                    },
                    "bulletPreset": "BULLET_CHECKBOX"
                }
            })
        else:
            # normal bullet
            requests.append({
                "createParagraphBullets": {
                    "range": {
                        "startIndex": p_start,
                        "endIndex": p_end
                    },
                    "bulletPreset": "BULLET_DISC_CIRCLE_SQUARE"
                }
            })
        # Optional: if you want indentation for nested lists,
        # you can handle indentation offsets. For simplicity, we skip that here.

    # If footer exists, style it
    if footer_started_index:
        requests.append({
            "updateTextStyle": {
                "range": {
                    "startIndex": footer_started_index,
                    "endIndex": current_index
                },
                "textStyle": {
                    "italic": True,
                    "foregroundColor": {
                        "color": {
                            "rgbColor": {"red": 0.5, "green": 0.5, "blue": 0.5}
                        }
                    }
                },
                "fields": "italic,foregroundColor"
            }
        })

    return requests

In [8]:
try:
    # Create a new Google Doc
    doc_title = "Product Team Sync (Automated)"
    doc = docs_service.documents().create(body={"title": doc_title}).execute()
    doc_id = doc.get("documentId")
    print("Created document with title:", doc_title)
    print("Document ID:", doc_id)
    print("Link:", f"https://docs.google.com/document/d/{doc_id}/edit")

    # Parse the Markdown
    elements = parse_markdown(markdown_text)

    # Build the batchUpdate requests
    requests = build_requests(elements)

    # Send the requests to Google Docs
    result = docs_service.documents().batchUpdate(
        documentId=doc_id,
        body={"requests": requests}
    ).execute()

    print("Successfully updated the Google Doc with parsed Markdown content.")
except Exception as e:
    print("Error creating/updating the document:", str(e))


Created document with title: Product Team Sync (Automated)
Document ID: 15GX0UTgiEu6PLx0x3tVnRMp25a4xpT5M8tw15TD9u90
Link: https://docs.google.com/document/d/15GX0UTgiEu6PLx0x3tVnRMp25a4xpT5M8tw15TD9u90/edit
Successfully updated the Google Doc with parsed Markdown content.
